In [ ]:
GITHUB_USERNAME = "paxtonfitzpatrick"#"$GITHUB_USERNAME$"
GITHUB_REF = "62cba0f23e89e5cb838ac84cd8ed88939bc4d784"#"$GITHUB_REF$"
NOTEBOOK_TYPE = "jupyter"#"$NOTEBOOK_TYPE$"
PYTHON_VERSION = "3.8"#"$PYTHON_VERSION$"
IPYTHON_VERSION = "latest"#"$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import builtins
import sys
import types
from contextlib import redirect_stdout
from io import StringIO
from pathlib import Path
from subprocess import CalledProcessError
from textwrap import dedent

import davos
import IPython
import pkg_resources
import tqdm
from IPython.utils.io import capture_output as capture_ipython_display

from utils import (
    expected_onion_parser_output, 
    expected_parser_output,
    is_imported, 
    is_installed,
    mark, 
    matches_expected_output,
    raises, 
    run_tests
)

In [ ]:
IPYTHON_SHELL = get_ipython()

_parse_onion = davos.core.core.Onion.parse_onion


def _parse_line(line):
    if IPYTHON_VERSION == 'latest' or int(IPYTHON_VERSION.split('.')[0]) >= 7:
        line = [f"{line}\n"]
    return davos.implementations.full_parser(line)

# tests for `davos.core.core`
**Note**: regular expressions defined in `davos.core.regexps` but used in `davos.core.core` are tested in `test_regexps.ipynb`

In [ ]:
def test_capture_stdout_multistream():
    """
    should write correct output to stdout (captured for test), 
    in-memory object, and file streams simultaneously
    """
    msg = ("Strictly speaking, I didn't do the theiving. That would be the "
           "pirates. I just moved what they stole from one place to another")
    tmpfile = Path('tmpfile.txt')
    try:
        with redirect_stdout(StringIO()) as mock_stdout:
            with davos.core.core.capture_stdout(
                StringIO(), tmpfile.open('w')
            ) as (mem_stream, file_stream):
                print(msg, end='')
                in_mem = mem_stream.getvalue()
            
            in_stdout = mock_stdout.getvalue()
        
        in_file = tmpfile.read_text()
        
        assert mem_stream.closed, "StringIO object was not closed"
        assert file_stream.closed, "file stream was not closed"

        assert in_stdout == msg, (
            "content written to stdout doesn't match original. Expected:\n"
            f"\"{msg}\"\n\nFound:\n\"{in_stdout}\""
        )
        assert in_mem == msg, (
            "content written to StringIO object doesn't match original. "
            f"Expected:\n\"{msg}\"\n\nFound:\n\"{in_stdout}\""
        )
        assert in_file == msg, (
            "content written to file doesn't match original. Expected:\n"
            f"\"{msg}\"\n\nFound:\n\"{in_file}\""
        )
    finally:
        if not mem_stream.closed:
            mem_stream.close()
        if not file_stream.closed:
            file_stream.close()
        if tmpfile.is_file():
            tmpfile.unlink()

In [ ]:
def test_capture_stdout_not_closing():
    """
    passing 'closing=True' should prevent streams from closing when 
    exiting context block
    """
    msg = ("Strictly speaking, I didn't do the theiving. That would be the "
           "pirates. I just moved what they stole from one place to another")
    try:
        with davos.core.core.capture_stdout(StringIO(), closing=False) as mem_stream:
            print(msg, end='')
                
        assert not mem_stream.closed, (
            "StringIO stream was closed despite passing 'closing=False'"
        )
    finally:
        mem_stream.close()

In [ ]:
@mark.ipython_pre7
def test_check_conda_ipython_pre7():
    """
    conda is not available on Google Colaboratory and not yet supported 
    for IPython<7.0.0
    """
    davos.core.core.check_conda()
    assert davos.config.conda_avail is False
    assert davos.config.conda_env is None
    assert davos.config.conda_envs_dirs is None

In [ ]:
@mark.ipython_post7
def test_check_conda_ipython_post7():
    """basic test for expected values"""
    davos.core.core.check_conda()
    assert davos.config.conda_avail is True
    assert davos.config.conda_env == 'kernel-env'
    assert isinstance(davos.config.conda_envs_dirs, dict)

In [ ]:
def test_check_conda_stdout_parsing():
    """
    check that 'conda_avail', 'conda_env', and 'conda_envs_dirs' config 
    fields are set correctly. Temporarily overloads 
    '_check_conda_avail_helper' and 'run_shell_command' functions with 
    functions defined below to compare expected values & results.
    """
    old_conda_avail = davos.config._conda_avail
    old_conda_env = davos.config._conda_env
    old_conda_envs_dirs = davos.config._conda_envs_dirs
    old__check_conda_avail_helper = davos.core.core._check_conda_avail_helper
    old_run_shell_command = davos.core.core.run_shell_command
    
    mock_conda_list_output = dedent("""\
    # packages in environment at /path/to/anaconda3/envs/mock-current-env:
    #
    # Name                    Version                   Build  Channel
    ipython                   X.X.X                    YYYYYY    ZZZZ
    etc...""")
    mock_conda_info_output = dedent("""\
    base /path/to/anaconda3
    mock-current-env /path/to/anaconda3/envs/mock-current-env
    mock-other-env /path/to/anaconda3/envs/mock-other-env""")
    
    expected_conda_envs_dirs = {
        'base': '/path/to/anaconda3',
        'mock-current-env': '/path/to/anaconda3/envs/mock-current-env',
        'mock-other-env': '/path/to/anaconda3/envs/mock-other-env'
    }
    
    def _mock_check_conda_avail_helper():
        return mock_conda_list_output
    
    def _mock_run_shell_command(command, live_stdout=None):
        if command == "conda info --envs | grep -E '^\w' | sed -E 's/ +\*? +/ /g'":
            return mock_conda_info_output
        else:
            return old_run_shell_command(command, live_stdout=live_stdout)
    
    try:
        davos.core.core._check_conda_avail_helper = _mock_check_conda_avail_helper
        davos.core.core.run_shell_command = _mock_run_shell_command
        
        davos.core.core.check_conda()
        
        assert davos.config.conda_avail is True
        assert davos.config.conda_env == 'mock-current-env', (
            f"Found: {davos.config.conda_env}"
        )
        assert davos.config.conda_envs_dirs == expected_conda_envs_dirs, (
            f"Expected: {expected_conda_envs_dirs}\nFound: "
            f"{davos.config.conda_envs_dirs}"
        )
    finally:
        davos.config._conda_avail = old_conda_avail
        davos.config._conda_env = old_conda_env
        davos.config._conda_envs_dirs = old_conda_envs_dirs
        davos.core.core._check_conda_avail_helper = old__check_conda_avail_helper
        davos.core.core.run_shell_command = old_run_shell_command

In [ ]:
def test_check_conda_bad_info_cmd():
    """
    when parsing 'conda list ipython' output is successful but parsing 
    'conda info --envs' output fails, 'conda_avail' and 'conda_env' 
    config fields should still be assigned correctly, but 
    'conda_envs_dirs' field should be 'None'
    """
    old_conda_avail = davos.config._conda_avail
    old_conda_env = davos.config._conda_env
    old_conda_envs_dirs = davos.config._conda_envs_dirs
    old__check_conda_avail_helper = davos.core.core._check_conda_avail_helper
    old_run_shell_command = davos.core.core.run_shell_command
    
    mock_conda_list_output = dedent("""\
    # packages in environment at /path/to/anaconda3/envs/mock-current-env:
    #
    # Name                    Version                   Build  Channel
    ipython                   X.X.X                    YYYYYY    ZZZZ
    etc...""")
    
    def _mock_check_conda_avail_helper():
        return mock_conda_list_output
    
    def _mock_run_shell_command(command, live_stdout=None):
        if command == "conda info --envs | grep -E '^\w' | sed -E 's/ +\*? +/ /g'":
            raise Exception
        else:
            return old_run_shell_command(command, live_stdout=live_stdout)
    
    try:
        davos.core.core._check_conda_avail_helper = _mock_check_conda_avail_helper
        davos.core.core.run_shell_command = _mock_run_shell_command
        
        davos.core.core.check_conda()
        
        assert davos.config.conda_avail is True
        assert davos.config.conda_env == 'mock-current-env', (
            f"Found: {davos.config.conda_env}"
        )
        assert davos.config.conda_envs_dirs is None
    finally:
        davos.config._conda_avail = old_conda_avail
        davos.config._conda_env = old_conda_env
        davos.config._conda_envs_dirs = old_conda_envs_dirs
        davos.core.core._check_conda_avail_helper = old__check_conda_avail_helper
        davos.core.core.run_shell_command = old_run_shell_command

In [ ]:
def test_check_conda_bad_list_cmd_raises():
    """
    when 'conda list ipython' output is parsed successfully, but yields 
    an environment name not in the dict parsed from the output of 
    'conda info --envs', 'check_conda' function should raise a 
    'DavosError'
    """
    old_conda_avail = davos.config._conda_avail
    old_conda_env = davos.config._conda_env
    old_conda_envs_dirs = davos.config._conda_envs_dirs
    old__check_conda_avail_helper = davos.core.core._check_conda_avail_helper
    old_run_shell_command = davos.core.core.run_shell_command
    
    mock_conda_list_output = dedent("""\
    # packages in environment at bad-env-path:
    #
    # Name                    Version                   Build  Channel
    ipython                   X.X.X                    YYYYYY    ZZZZ
    etc...""")
    mock_conda_info_output = dedent("""\
    base /path/to/anaconda3
    mock-current-env /path/to/anaconda3/envs/mock-current-env
    mock-other-env /path/to/anaconda3/envs/mock-other-env""")
    
    expected_conda_envs_dirs = {
        'base': '/path/to/anaconda3',
        'mock-current-env': '/path/to/anaconda3/envs/mock-current-env',
        'mock-other-env': '/path/to/anaconda3/envs/mock-other-env'
    }
    
    def _mock_check_conda_avail_helper():
        return mock_conda_list_output
    
    def _mock_run_shell_command(command, live_stdout=None):
        if command == "conda info --envs | grep -E '^\w' | sed -E 's/ +\*? +/ /g'":
            return mock_conda_info_output
        else:
            return old_run_shell_command(command, live_stdout=live_stdout)
    
    try:
        davos.core.core._check_conda_avail_helper = _mock_check_conda_avail_helper
        davos.core.core.run_shell_command = _mock_run_shell_command
        
        with raises(davos.core.exceptions.DavosError):
            davos.core.core.check_conda()

    finally:
        davos.config._conda_avail = old_conda_avail
        davos.config._conda_env = old_conda_env
        davos.config._conda_envs_dirs = old_conda_envs_dirs
        davos.core.core._check_conda_avail_helper = old__check_conda_avail_helper
        davos.core.core.run_shell_command = old_run_shell_command

In [ ]:
def test_get_previously_imported_pkgs():
    """
    should return names of packages imported during this interpreter 
    session, whether from within the current notebook (davos, requests)
    or elsewhere (IPython, urllib3), but ignore all other names 
    (scikit-learn, selenium). Uses mock 'pip-install'-like output to set 
    up expected results. Tests against real output in 
    'test_regexps.ipynb'.
    """
    mock_stdout = ("Successfully installed davos-0.0.0 IPython-1.1.1 "
              "requests-2.2.2 scikit-learn-3.3.3 selenium-4.4.4 urllib3-5.5.5")
    expected = ['davos', 'IPython', 'requests', 'urllib3']
    result = davos.core.core.get_previously_imported_pkgs(mock_stdout, 'pip')
    assert result == expected, f"Expected:\n'{expected}'\nFound:\n'{result}'"

In [ ]:
def test_get_previously_imported_pkgs_conda_raises():
    """passing 'installer-"conda"' should raise 'NotImplementedError'"""
    with raises(NotImplementedError):
        davos.core.core.get_previously_imported_pkgs('', 'conda')

In [ ]:
def test_import_name_pkg():
    """test importing a top-level package."""
    try:
        assert not is_imported('scipy')
        scipy = davos.core.core.import_name('scipy')
        assert isinstance(scipy, types.ModuleType), (
            f"type(scipy): {type(scipy)}\nMRO: {type(scipy).mro()}"
        )
        assert not hasattr(scipy, 'stats'), (
            "scipy.stats submodule should not be loaded"
        )
        assert 'scipy' in sys.modules
    finally:
        for mod_name in tuple(sys.modules.keys()):
            if 'scipy' in mod_name:
                sys.modules.pop(mod_name)

In [ ]:
def test_import_name_module():
    """test importing a module from a package."""
    try:
        assert not is_imported('scipy')
        stats = davos.core.core.import_name('scipy.stats')
        assert isinstance(stats, types.ModuleType), (
            f"type(stats): {type(stats)}\nMRO: {type(stats).mro()}"
        )
        assert not hasattr(stats, 'tests'), (
            "scipy.stats.tests submodule should not be loaded"
        )
        assert 'scipy' in sys.modules
        assert 'scipy.stats' in sys.modules
    finally:
        for mod_name in tuple(sys.modules.keys()):
            if 'scipy' in mod_name:
                sys.modules.pop(mod_name)

In [ ]:
def test_import_name_function():
    """test importing a function by its qualified name"""
    try:
        assert not is_imported('scipy')
        ttest_ind = davos.core.core.import_name('scipy.stats.ttest_ind')
        assert isinstance(ttest_ind, types.FunctionType), (
            f"type(scipy): {type(ttest_ind)}\nMRO: {type(ttest_ind).mro()}"
        )
        assert ttest_ind.__module__ == 'scipy.stats.stats', (
            f"Expected: 'scipy.stats.stats'\nFound: '{ttest_ind.__module__}'"
        )
        assert 'scipy' in sys.modules
        assert 'scipy.stats' in sys.modules
        assert 'scipy.stats.stats' in sys.modules
    finally:
        for mod_name in tuple(sys.modules.keys()):
            if 'scipy' in mod_name:
                sys.modules.pop(mod_name)

In [ ]:
def test_parse_onion_simple():
    """simplest case"""
    onion = '# pip: foo==0.0.1'
    expected = expected_onion_parser_output('foo==0.0.1')
    result = _parse_onion(onion)
    assert result == expected, f"Expected:\n{expected}\nResult:\n{result}"

In [ ]:
def test_parse_onion_conda_raises():
    """installing with conda is not yet supported"""
    onion = '# conda: foo==0.0.1'
    with raises(davos.core.exceptions.ParserNotImplementedError):
        _parse_onion(onion)

In [ ]:
def test_parse_onion_bad_installer_raises():
    """should throw an error if an unrecognized installer is specified"""
    onion = '# apt: foo==0.0.1'
    with raises(davos.core.exceptions.OnionParserError):
        _parse_onion(onion)

In [ ]:
def test_parse_onion_whitespace():
    """onion comment should be whitespace-insensitive"""
    onion = '#             pip:            foo==0.0.1'
    expected = expected_onion_parser_output('foo==0.0.1')
    result = _parse_onion(onion)
    assert result == expected, f"Expected:\n{expected}\nResult:\n{result}"

In [ ]:
def test_parse_onion_github():
    """test parsing a VCS (specifically GitHub) url"""
    onion = '# pip: git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz'
    expected = expected_onion_parser_output('git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz')
    result = _parse_onion(onion)
    assert result == expected, f"Expected:\n{expected}\nResult:\n{result}"

In [ ]:
def test_parse_onion_editable():
    """GitHub URL like above, but with the --editable flag supplied"""
    onion = '# pip: --editable git+https://github.com/foo/bar.git'
    expected = expected_onion_parser_output('--editable git+https://github.com/foo/bar.git', 
                                            editable=True, 
                                            spec='git+https://github.com/foo/bar.git')
    result = _parse_onion(onion)
    assert result == expected, f"Expected:\n{expected}\nResult:\n{result}"

In [ ]:
def test_parse_onion_joined_short_args():
    onion = '# pip: -Ive git+https://github.com/foo/bar.git'
    expected = expected_onion_parser_output('-Ive git+https://github.com/foo/bar.git', 
                                            editable=True, 
                                            ignore_installed=True, 
                                            verbosity=1,
                                            spec='git+https://github.com/foo/bar.git')
    result = _parse_onion(onion)
    assert result == expected, f"Expected:\n{expected}\nResult:\n{result}"

In [ ]:
def test_onion_simple():
    """
    simple Onion object created from 'smuggle' statement with no onion 
    comment.
        `smuggle foo`
    """
    onion = davos.core.core.Onion('foo', installer='pip', args_str="""""")
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;', (
        f"Expected:\npip;\nFound:\n{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == 'foo', (
        f"Expected\nfoo\nFound:\n{onion.install_name}"
    )
    assert onion.version_spec == '', (
        f"Expected: ''\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_simple_onion():
    """
    like above, but with a relatively simple onion comment
        `smuggle foo    # pip: foo==0.0.1 -vv`
    """
    installer_kwargs = {
        'editable': False, 
        'spec': 'foo==0.0.1', 
        'verbosity': 2
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""foo==0.0.1 -vv""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;foo==0.0.1;-vv', (
        f"Expected:\npip;foo==0.0.1;-vv\nFound:\n{onion.cache_key}"
    )
    assert onion.verbosity == 2, f"Expected: 2\nFound: {onion.verbosity}"
    assert onion.install_name == 'foo', (
        f"Expected\nfoo\nFound:\n{onion.install_name}"
    )
    assert onion.version_spec == '==0.0.1', (
        f"Expected: '==0.0.1'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_different_install_name():
    """
    a package imported under a different name than is used to install it
        `smuggle foo    # pip: --editable foo-pkg==0.0.1`
    """
    installer_kwargs = {
        'editable': True, 
        'spec': 'foo-pkg==0.0.1'
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""--editable foo-pkg==0.0.1""", 
                                  **installer_kwrags)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;--editable;foo-pkg==0.0.1', (
        f"Expected:\npip;--editable;foo==0.0.1\nFound:\n{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 2\nFound: {onion.verbosity}"
    assert onion.install_name == 'foo-pkg', (
        f"Expected\nfoo-pkg\nFound:\n{onion.install_name}"
    )
    assert onion.version_spec == '==0.0.1', (
        f"Expected: '==0.0.1'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_conda_raises():
    """passing 'installer="conda"' should raise NotImplementedError"""
    with raises(NotImplementedError):
        onion = davos.core.core.Onion('foo', installer='conda', args_str="""""")

In [ ]:
def test_onion_bad_installer_raises():
    """should throw an error if passed an unrecognized installer"""
    with raises(davos.core.exceptions.InstallerError):
        onion = davos.core.core.Onion('foo', installer='apt', args_str="""""")

In [ ]:
def test_onion_vcs_simple():
    """
    relatively simple VCS (GitHub) URL without modifiers
        `smuggle bar    # pip: git+https://github.com/foo/bar.git`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'git+https://github.com/foo/bar.git'
    }
    onion = davos.core.core.Onion('bar', 
                                  installer='pip', 
                                  args_str="""git+https://github.com/foo/bar.git""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;git+https://github.com/foo/bar.git', (
        "Expected:\npip;git+https://github.com/foo/bar.git\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == 'git+https://github.com/foo/bar.git', (
        "Expected\ngit+https://github.com/foo/bar.git\nFound:\n"
        f"{onion.install_name}"
    )
    assert onion.version_spec == '', (
        f"Expected: ''\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_vcs_ref():
    """
    same as above, but the GitHub URL contains a reference to a specific 
    revision (commit, branch, tag, etc.)
        `smuggle bar    # pip: git+https://github.com/foo/bar.git@ref`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'git+https://github.com/foo/bar.git@ref'
    }
    onion = davos.core.core.Onion('bar', 
                                  installer='pip', 
                                  args_str="""git+https://github.com/foo/bar.git@ref""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;git+https://github.com/foo/bar.git@ref', (
        "Expected:\npip;git+https://github.com/foo/bar.git@ref\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == 'git+https://github.com/foo/bar.git', (
        "Expected\ngit+https://github.com/foo/bar.git\nFound:\n"
        f"{onion.install_name}"
    )
    assert onion.version_spec == '@ref', (
        f"Expected: '@ref'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_vcs_ref_egg():
    """
    same as above, but the GitHub URL also specifies the project name 
    via the '#egg=' component of the URL
        `smuggle bar    # pip: git+https://github.com/foo/bar.git@ref#bar`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'git+https://github.com/foo/bar.git@ref#bar'
    }
    onion = davos.core.core.Onion('bar', 
                                  installer='pip', 
                                  args_str="""git+https://github.com/foo/bar.git@ref#bar""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;git+https://github.com/foo/bar.git@ref#bar', (
        "Expected:\npip;git+https://github.com/foo/bar.git@ref#bar\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == 'git+https://github.com/foo/bar.git#bar', (
        "Expected\ngit+https://github.com/foo/bar.git#bar\nFound:\n"
        f"{onion.install_name}"
    )
    assert onion.version_spec == '@ref', (
        f"Expected: '@ref'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_vcs_ref_egg_subdir():
    """
    same as above, but the GitHub URL also specifies the subdirectory 
    path of the package within the repository via the '&subdirectory=' 
    component of the URL
        `smuggle bar    # pip: git+https://github.com/foo/bar.git@ref#bar&subdirectory=/baz/qux`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'git+https://github.com/foo/bar.git@ref#bar'
    }
    onion = davos.core.core.Onion('bar', 
                                  installer='pip', 
                                  args_str="""git+https://github.com/foo/bar.git@ref#bar""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;git+https://github.com/foo/bar.git@ref#bar&subdirectory=/baz/qux', (
        "Expected:\npip;git+https://github.com/foo/bar.git@ref#bar&subdirectory=/baz/qux\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == 'git+https://github.com/foo/bar.git#bar&subdirectory=/baz/qux', (
        "Expected\ngit+https://github.com/foo/bar.git#bar\nFound:\n"
        f"{onion.install_name}"
    )
    assert onion.version_spec == '@ref', (
        f"Expected: '@ref'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_local():
    """
    package installed (in editable mode) from a local directory
        `smuggle foo    # pip: -e /path/to/foo`
    """
    installer_kwargs = {
        'editable': True,
        'spec': '/path/to/foo'
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""-e /path/to/foo""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;-e;/path/to/foo', (
        "Expected:\npip;-e;/path/to/foo\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 0, f"Expected: 0\nFound: {onion.verbosity}"
    assert onion.install_name == '/path/to/foo', (
        "Expected\n/path/to/foo\nFound:\n"
        f"{onion.install_name}"
    )
    assert onion.version_spec == '', (
        f"Expected: ''\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_pep440_direct_reference():
    """
    package installed from a remote source archive file. See 
    https://www.python.org/dev/peps/pep-0440/
        `smuggle foo    # pip: -v foo@https://bar.repo/foo-0.0.1-non-any.whl`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'foo@https://bar.repo/foo-0.0.1-non-any.whl',
        'verbosity': 1
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""-v foo@https://bar.repo/foo-0.0.1-non-any.whl""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;-v;foo@https://bar.repo/foo-0.0.1-non-any.whl', (
        "Expected:\npip;-v;foo@https://bar.repo/foo-0.0.1-non-any.whl\nFound:\n"
        f"{onion.cache_key}"
    )
    assert onion.verbosity == 1, f"Expected: 1\nFound: {onion.verbosity}"
    assert onion.install_name == 'foo@https://bar.repo/foo-0.0.1-non-any.whl', (
        "Expected\nfoo@https://bar.repo/foo-0.0.1-non-any.whl\nFound:\n{onion.install_name}"
    )
    assert onion.version_spec == '', (
        f"Expected: ''\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_complex_version():
    """
    onion comment that constrains the smuggled package version using 
    multiple specifiers. -v/--verbose are passed in a somewhat 
    unrealistic way, but it's meant to test isolating the version 
    specifier from other arugments both before and after it
        `smuggle foo    # pip: -v foo-pkg>=1.0.1,<2 --verbose`
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'foo-pkg>=1.0.1,<2',
        'verbosity': 2
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""-v foo-pkg>=1.0.1,<2 --verbose""", 
                                  **installer_kwargs)
    assert onion.install_package == onion._pip_install_package, (
        f"Expected:\n{onion._pip_install_package}\nFound:\n"
        f"{onion.install_package}"
    )
    assert onion.build is None, f"Expected: None\nFound: {onion.build}"
    assert onion.cache_key == 'pip;foo-pkg>=1.0.1,<2;--verbose', (
        "Expected:\npip;foo-pkg>=1.0.1,<2;--verbose\nFound:\n{onion.cache_key}"
    )
    assert onion.verbosity == 1, f"Expected: 1\nFound: {onion.verbosity}"
    assert onion.install_name == 'foo-pkg', (
        f"Expected\nfoo-pkg\nFound:\n{onion.install_name}"
    )
    assert onion.version_spec == '>=1.0.1,<2', (
        f"Expected: '>=1.0.1,<2'\nFound: {onion.version_spec}"
    )

In [ ]:
def test_onion_is_installed_stdlib():
    """
    Onion.is_installed property should always return True for standard 
    library modules
    """
    test_mods = ('ast', 'base64', 'collections', 'datetime', 'enum')
    for mod in test_mods:
        onion = davos.core.core.Onion(mod, installer='pip', args_str="""""")
        assert onion.is_installed, (
            f"Expected standard library module '{mod}' to be installed"
        )

In [ ]:
def test_onion_is_installed_args_false():
    """
    Onion.is_installed property should always return False when these 
    arguments are supplied. Test using a package that is definitely 
    installed to make sure the presence of these arguments is actually 
    responsible for the False return value
    """
    always_false_args = ('--force-reinstall', '--ignore-installed', '--upgrade')
    for arg in always_false_args:
        kwarg_form = arg.lstrip('-').replace('-', '_')
        installer_kwargs = {
            'editable': False, 
            kwarg_form: True, 
            'spec': 'IPython'
        }
        onion = davos.core.core.Onion('IPython', 
                                      installer='pip', 
                                      args_str=f"""{arg} IPython""", 
                                      **installer_kwargs)
        assert not onion.is_installed, (
            f"Passing {arg} should cause onion.is_installed to return False"
        )

In [ ]:
def test_onion_is_installed_vcs_false():
    """
    Onion.is_installed property should always return False when the 
    onion comment specifies a VCS URL, since there's no way to compare 
    the state of a local installation to the remote repository (e.g., by 
    commit hash). Like above, test using a package that's definitely 
    installed, for which onion.is_installed would otherwise be True.
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'git+https://github.com/ipython/ipython.git'
    }
    onion = davos.core.core.Onion('bar', 
                                  installer='pip', 
                                  args_str="""git+https://github.com/ipython/ipython.git""", 
                                  **installer_kwargs)
    
    assert not onion.is_installed, (
        "specifying a VCS URL should always cause onion.is_installed to "
        "return False"
    )

In [ ]:
def test_onion_is_installed_nonmatching_package():
    """
    Test result of Onion.is_installed searching installed package 
    distributions for a package that *IS NOT* installed locally
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'FakePackage'
    }
    onion = davos.core.core.Onion('FakePackage', 
                                  installer='pip', 
                                  args_str="""""", 
                                  **installer_kwargs)
    
    assert not onion.is_installed, (
        "Should not have found local installation of 'FakePackage'"
    )

In [ ]:
def test_onion_is_installed_matching_package():
    """
    Test result of Onion.is_installed searching installed package 
    distributions for a package that *IS* installed locally, with no 
    version constraints
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'IPython'
    }
    onion = davos.core.core.Onion('IPython', 
                                  installer='pip', 
                                  args_str="""IPython""", 
                                  **installer_kwargs)
    
    assert onion.is_installed, "Failed to find local IPython installation"

In [ ]:
def test_onion_is_installed_nonmatching_version():
    """
    Test result of Onion.is_installed searching installed package 
    distributions for a package that *IS* installed locally, but whose 
    local installation *DOES NOT* satisfy the requested version 
    constraint
    """
    installer_kwargs = {
        'editable': False,
        'spec': 'IPython<=2.3.4'
    }
    onion = davos.core.core.Onion('IPython', 
                                  installer='pip', 
                                  args_str="""IPython<=2.3.4""", 
                                  **installer_kwargs)
    
    assert not onion.is_installed, (
        "Should not have found local installation satisfying "
        f"'IPython<=2.3.4'. Local version is '{IPython.__version__}'"
    )

In [ ]:
def test_onion_is_installed_matching_version():
    """
    Test result of Onion.is_installed searching installed package 
    distributions for a package that *IS* installed locally, and whose 
    local installation *DOES* satisfy the requested version constraint
    """
    
    installer_kwargs = {
        'editable': False,
        'spec': f'IPython=={IPython.__version__}'
    }
    onion = davos.core.core.Onion('IPython', 
                                  installer='pip', 
                                  args_str=f"""IPython=={IPython.__version__}""", 
                                  **installer_kwargs)
    
    assert onion.is_installed, (
        "Failed to find local IPython installation matching locally "
        f"installed version ({IPython.__version__})"
    )

In [ ]:
def test_pip_install_package_fail():
    """test handling of install shell command that fails"""
    onion = davos.core.core.Onion('foo', installer='pip', args_str="""""")
    
    expected_returncode = 1
    expected_cmd = onion.install_cmd
    expected_output = "Expected stdout string"
    expected_stderr = "Expected stderr string"
    
    old_run_shell_command = davos.core.core.run_shell_command
    
    def _mock_run_shell_command(command, live_stdout=None):
        if command == onion.install_cmd:
            raise CalledProcessError(returncode=expected_returncode, 
                                     cmd=expected_cmd, 
                                     output=expected_output, 
                                     stderr=expected_stderr)
        else:
            return old_run_shell_command(command, live_stdout=live_stdout)
        
    try:
        davos.core.core.run_shell_command = _mock_run_shell_command
        with raises(davos.core.exceptions.InstallerError) as excinfo:
            onion._pip_install_package
        
        exc_value = excinfo.value
        result_returncode = exc_value.returncode
        result_cmd = exc_value.cmd
        result_output = exc_value.output
        result_stderr = exc_value.stderr
        
        assert result_returncode == expected_returncode, result_returncode
        assert result_cmd == expected_cmd, result_cmd
        assert result_output == expected_output, result_output
        assert result_stderr == expected_stderr, result_stderr
        
    finally:
        davos.core.core.run_shell_command = old_run_shell_command

In [ ]:
def test_pip_install_package_target_prepend_syspath():
    """
    when installing a package in a specified --target directory that is 
    not already in sys.path, the directory should be prepended to 
    sys.path
    """
    old_run_shell_command = davos.core.core.run_shell_command
    old_syspath = sys.path[:]
    tmpdir = Path('tmpdir').resolve()
    installer_kwargs = {
        'editable': False,
        'spec': 'foo',
        'target': 'tmpdir'
    }
    onion = davos.core.core.Onion('foo', 
                                  installer='pip', 
                                  args_str="""foo --target tmpdir""", 
                                  **installer_kwargs)
    mock_stdout = "stdout from pip-installing 'foo' in tmpdir/"
    
    def _mock_run_shell_command(command, live_stdout=None):
        if command == onion.install_cmd:
            return mock_stdout
        else:
            return old_run_shell_command(command, live_stdout=live_stdout)

    try:
        tmpdir.mkdir()
        assert str(tmpdir) not in sys.path, f"{tmpdir} already in sys.path"
        onion._pip_install_package()
        assert sys.path[0] == str(tmpdir), (
            f"{tmpdir} was not prepended to sys.path\nsys.path:\n{sys.path}")
        
    finally:
        davos.core.core.run_shell_command = old_run_shell_command
        sys.path = old_syspath
        if tmpdir.is_dir():
            tmpdir.rmdir()

In [ ]:
def test_parser_handles_basic_line():
    """simplest use case"""
    line = "smuggle foo"
    expected = expected_parser_output('foo')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_basic_line_alias():
    """simplest use case, plus alias"""
    line = "smuggle foo as bar"
    expected = expected_parser_output('foo', as_='bar')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_basic_line_onion():
    """simplest use case, with onion comment"""
    line = "smuggle foo as bar    # pip: foo==0.0.1"
    expected = expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_qualname():
    line = "smuggle foo.bar as baz"
    expected = expected_parser_output('foo.bar', as_='baz')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_indented_line():
    """
    needs to handle, e.g.
        ```
        def foo():
            smuggle numpy as np
        ```
    Note: unless `davos.deactivate() is run, the above line will 
    actually be parsed (though it will have no effect)
    """
    line = "    smuggle foo as bar"
    expected = "    " + expected_parser_output('foo', as_='bar')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_different_install_name():
    """different names used to install and import package"""
    line = "smuggle foo as bar    # pip: foo-package==0.0.1"
    expected = expected_parser_output('foo', as_='bar', 
                                       args_str='foo-package==0.0.1')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_multiple_packages():
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs"
    expected = expected_parser_output('foo', as_='bar')
    expected += f"; {expected_parser_output('baz', as_='qux')}"
    expected += f"; {expected_parser_output('spam')}"
    expected += f"; {expected_parser_output('ham')}"
    expected += f"; {expected_parser_output('eggs')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_multiple_packages_onion():
    """
    onion info should be passed to smuggle function for FIRST package
    """
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs    # pip: foo==0.0.1"
    expected = expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('baz', as_='qux')}"
    expected += f"; {expected_parser_output('spam')}"
    expected += f"; {expected_parser_output('ham')}"
    expected += f"; {expected_parser_output('eggs')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_backslash():
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs"""
    expected = expected_parser_output('foo', as_='bar')
    expected += f"; {expected_parser_output('baz', as_='qux')}"
    expected += f"; {expected_parser_output('spam')}"
    expected += f"; {expected_parser_output('ham')}"
    expected += f"; {expected_parser_output('eggs')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_backslash_onion():
    """
    again, onion info should be passed to smuggle function for FIRST 
    package
    """
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs    # pip: foo==0.0.1"""
    expected = expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('baz', as_='qux')}"
    expected += f"; {expected_parser_output('spam')}"
    expected += f"; {expected_parser_output('ham')}"
    expected += f"; {expected_parser_output('eggs')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_inconsistent_whitespace():
    """should handle weird amounts of whitespace that are *technically* valid"""
    line = """smuggle               foo     as    bar    \
,baz as qux  , \
          spam  .  ham    as    eggs                #   pip   :       foo==0.0.1    """
    expected = expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('baz', as_='qux')}"
    expected += f"; {expected_parser_output('spam.ham', as_='eggs')}"
    # real parser adds back leading & trailing characters (including whitespace)
    expected += "    "
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from():
    """second possible broad syntax class"""
    line = "from foo smuggle bar"
    expected = expected_parser_output('foo.bar', as_='bar')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_onion():
    """second possible broad syntax class"""
    line = "from foo smuggle bar    # pip: foo==0.0.1"
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_multi():
    line = "from foo smuggle bar, baz as spam, qux"
    expected = expected_parser_output('foo.bar', as_='bar')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_multi_onion():
    """onion info should be passed to FIRST smuggle function"""
    line = "from foo smuggle bar, baz as spam, qux    # pip: foo==0.0.1"
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_backslash():
    """onion info should be passed to FIRST smuggle function"""
    line = """from foo smuggle bar, \
                               baz as spam, \
                               qux    # pip: foo==0.0.1"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses():
    line = """from foo smuggle (bar, baz as spam, qux,)"""
    # also tests trailing comma inside parentheses, which is valid
    expected = expected_parser_output('foo.bar', as_='bar')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_1():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)"""
    expected = expected_parser_output('foo.bar', as_='bar')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_1():
    line = """from foo smuggle (bar,    # pip: foo==0.0.1
                                baz as spam, 
                                qux)"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_2():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)    # pip: foo==0.0.1"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = expected_parser_output('foo.bar', as_='bar')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_1():
    line = """from foo smuggle (    # pip: foo==0.0.1
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )    # pip: foo==0.0.1"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2_comments():
    line = """from foo smuggle (    # unrelated comment on first line
                  bar,    # unrelated comment on package name line
                  baz as spam, 
                  # unrelated comment on its own line
                  qux
              )    # pip: foo==0.0.1    # unrelated comment after onion"""
    expected = expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {expected_parser_output('foo.qux', as_='qux')}"
    expected += "    # unrelated comment after onion"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_parser_handles_smuggle_semicolons():
    """also combines multiple elements from prior tests"""
    line = """smuggle foo; smuggle bar as baz; \
              from spam smuggle eggs; \
              from qux smuggle quux as corge    # pip: qux-package==0.0.1"""
    expected = expected_parser_output('foo')
    expected += f"; {expected_parser_output('bar', as_='baz')}"
    expected += f"; {expected_parser_output('spam.eggs', as_='eggs')}"
    expected += f"; {expected_parser_output('qux.quux', as_='corge', args_str='qux-package==0.0.1')}"
    assert matches_expected_output(expected, _parse_line(line))

In [ ]:
def test_prompt_input_yes():
    """mock user input of 'yes'"""
    prompt_text = ''
    
    def _mock_input(prompt=None):
        nonlocal prompt_text
        prompt_text = prompt
        return 'yes'
    
    try:
        davos.core.core.input = _mock_input
        result = davos.core.core.prompt_input('prompt question?')
        assert result is True, f"Expected: True\nFound: {result}"
        assert prompt_text.strip().endswith('[y/n]'), (
            f"Expected prompt to end with '[y/n]'\nFound: {prompt_text}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_no():
    """mock user input of 'no'"""
    prompt_text = ''
    
    def _mock_input(prompt=None):
        nonlocal prompt_text
        prompt_text = prompt
        return 'no'
    
    try:
        davos.core.core.input = _mock_input
        result = davos.core.core.prompt_input('prompt question?')
        assert result is False, f"Expected: False\nFound: {result}"
        assert prompt_text.strip().endswith('[y/n]'), (
            f"Expected prompt to end with '[y/n]'\nFound: {prompt_text}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_default_accept():
    """
    mock user pressing return without entering any text when default 
    value *IS* set
    """
    prompt_text = ''
    
    def _mock_input(prompt=None):
        nonlocal prompt_text
        prompt_text = prompt
        return ''
    
    try:
        davos.core.core.input = _mock_input
        result = davos.core.core.prompt_input('prompt question?', default='y')
        assert result is True, f"Expected: True\nFound: {result}"
        assert prompt_text.strip().endswith('[Y/n]'), (
            f"Expected prompt to end with '[Y/n]'\nFound: {prompt_text}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_default_reject():
    """
    mock user pressing return without entering any text, when default 
    value *IS NOT* set
    """
    prompt_text = ''
    inputs = ['', '', '', 'y']
    input_ix = 0
    
    def _mock_input(prompt=None):
        nonlocal prompt_text, input_ix
        prompt_text = prompt
        return inputs[input_ix]
    
    try:
        davos.core.core.input = _mock_input
        result = davos.core.core.prompt_input('prompt question?', default=None)
        assert result is True, f"Expected: True\nFound: {result}"
        assert prompt_text.strip().endswith('[y/n]'), (
            f"Expected prompt to end with '[y/n]'\nFound: {prompt_text}"
        )
        assert input_ix == 3, (
            "Expected to cycle through empty inputs until reaching non-empty "
            f"string. Input was accepted when input_ix=={input_ix}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_default_invalid():
    """
    should raise ValueError if the value passed to 'default' is not 
    valid
    """
    def _mock_input(prompt=None):
        return 'y'
    try:
        davos.core.core.input = _mock_input
        with raises(ValueError)
            davos.core.core.prompt_input('prompt question?', default='Q')
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_interrupt_accept():
    """
    mock user pressing raising KeyboardInterrupt when interrupt value 
    *IS* set
    """
    prompt_text = ''
    
    def _mock_input(prompt=None):
        nonlocal prompt_text
        prompt_text = prompt
        raise KeyboardInterrupt()
    
    try:
        davos.core.core.input = _mock_input
        result = davos.core.core.prompt_input('prompt question?', 
                                              default='no', 
                                              interrupt='y')
        assert result is True, f"Expected: True\nFound: {result}"
        assert prompt_text.strip().endswith('[y/N]'), (
            f"Expected prompt to end with '[y/N]'\nFound: {prompt_text}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_interrupt_raise():
    """
    mock user pressing raising KeyboardInterrupt when interrupt value 
    *IS NOT* set
    """
    prompt_text = ''
    
    def _mock_input(prompt=None):
        nonlocal prompt_text
        prompt_text = prompt
        raise KeyboardInterrupt()
    
    try:
        davos.core.core.input = _mock_input
        with raises(KeyboardInterrupt):
            result = davos.core.core.prompt_input('prompt question?', 
                                                  default='yes')
        assert prompt_text.strip().endswith('[Y/n]'), (
            f"Expected prompt to end with '[Y/n]'\nFound: {prompt_text}"
        )
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_prompt_input_interrupt_invalid():
    """
    should raise ValueError if the value passed to 'interrupt' is not 
    valid
    """
    def _mock_input(prompt=None):
        return 'y'
    try:
        davos.core.core.input = _mock_input
        with raises(ValueError)
            davos.core.core.prompt_input('prompt question?', interrupt='Z')
    finally:
        davos.core.core.input = builtins.input

In [ ]:
def test_run_shell_command_simple():
    stdout = davos.core.core.run_shell_command('whoami', live_stdout=False)
    stdout = stdout.strip()
    assert stdout == 'root', f"Expected: root\nFound: {stdout}"

In [ ]:
def test_run_shell_command_multiword():
    quote = ("Strictly speaking, I didn't do the theiving. That would be the "
             "pirates. I just moved what they stole from one place to another")
    stdout = davos.core.core.run_shell_command(f'echo "{quote}"', 
                                               live_stdout=False)
    stdout = stdout.strip()
    assert stdout == quote, f'Expected:\n"{quote}"\nFound:\n"{stdout}"'

In [ ]:
def test_run_shell_command_failure():
    expected_output = '/bin/bash: blahblahblah: command not found'
    with raises(CalledProcessError) as excinfo:
        davos.core.core.run_shell_command('/bin/bash -c "blahblahblah"', 
                                          live_stdout=True)
    retcode = excinfo.value.returncode
    output = excinfo.value.output.strip()
    assert retcode == 127, f"Expected return code 127, found {retcode}"
    assert output == expected_output, (
        f'Expected output: "{expected_output}"\nFound: "{output}"'
    )

In [ ]:
@mark.timeout(30)
def test_smuggle_pip_new():
    """
    smuggle a package that does't exist locally, using 'pip' as the 
    installer program
    """
    assert not is_installed('ppca')
    smuggle ppca    # pip: ppca>=0.0.4
    assert isinstance(ppca, types.ModuleType), (
        "Expected smuggled 'ppca' object to be a module. type(ppca) is "
        f"{type(ppca)}"
    )
    assert hasattr(ppca, 'PPCA')
    assert is_installed('ppca>=0.0.4'), (
        "installed 'ppca' version does not match version specified in onion "
        "comment"
    )
    assert davos.config.smuggled['ppca'] == 'pip;ppca>=0.0.4', (
        f"Expected: 'pip;ppca>=0.0.4'\nFound: {davos.config.smuggled['ppca']}"
    )

In [ ]:
@mark.timeout(30)
def test_smuggle_from_pip_new():
    """
    smuggle a single name from a package that doesn't exist locally, 
    using 'pip' as the installer program
    """
    assert not is_installed('umap-learn')
    from umap smuggle UMAP    # pip: umap-learn==0.4.6
    assert hasattr(UMAP, 'fit'), (
        "Expected 'umap.UMAP' object to have attribute 'fit'. type(UMAP) is "
        f"{type(UMAP)}"
    )
    assert is_installed('umap-learn==0.4.6'), (
        "installed 'umap-learn' version does not match version specified in "
        "onion commetn"
    )
    assert davos.config.smuggled['umap-learn'] == 'pip;umap-learn==0.4.6', (
        "Expected: 'pip;umap-learn==0.4.6'\nFound: "
        f"{davos.config.smuggled['umap-learn']}"
    )

In [ ]:
@mark.timeout(30)
def test_smuggle_previously_installed():
    """
    smuggle a package that was already installed locally, but whose 
    local version does not match the requested version
    """
    assert is_installed('fastdtw==0.3.4')
    assert not is_imported('fastdtw')
    smuggle fastdtw    # pip: fastdtw==0.3.2
    assert isinstance(fastdtw, types.ModuleType), (
        "Expected smuggled 'fastdtw' object to be a module. type(fastdtw) is "
        f"{type(fastdtw)}"
    )
    assert is_installed('fastdtw==0.3.2')
    assert davos.config.smuggled['fastdtw'] == 'pip;fasdtw==0.3.2', (
        "Expected: 'pip;fasdtw==0.3.2'\nFound: "
        f"{davos.config.smuggled['fastdtw']}"
    )

In [ ]:
@mark.timeout(30)
def test_smuggle_previously_imported():
    """
    smuggle a new version of a package that was already imported during 
    the current interpreter session
    """
    assert tqdm.__version__ == '4.41.1'
    smuggle tqdm    # pip: tqdm==4.45.0
    assert tqdm.__version__ == '4.45.0'
    assert davos.config.smuggled['tqdm'] == 'pip;tqdm==4.45.0', (
        f"Expected: 'pip;tqdm==4.45.0'\nFound: {davos.config.smuggled['tqdm']}"
    )

In [ ]:
@mark.timeout(60)
def test_smuggle_github_ref():
    """
    smuggle a specific VCS revision of a package from GitHub
    Note: commit hash corresponds to v0.7.0 release
    """
    assert not is_installed('hypertools')
    smuggle hypertools as hyp    # pip: git+https://github.com/ContextLab/hypertools.git@e7b7446#egg=hypertools
    assert isinstance(hyp, types.ModuleType)
    assert hyp is sys.modules['hypertools']
    assert hasattr(hyp, 'DataGeometry')
    assert hyp.__version__ == '0.7.0'
    assert is_installed('hypertools==0.7.0')
    expected_cache_key = 'pip;git+https://github.com/ContextLab/hypertools.git@e7b7446#egg=hypertools'
    assert davos.config.smuggled['hypertools'] == expected_cache_key, (
        f"Expected: '{expected_cache_key}'\nFound: "
        f"{davos.config.smuggled['hypertools']}"
    )

In [ ]:
@mark.timeout(30)
def test_smuggle_github_editable():
    """
    smuggle a package from a VCS URL in 'editable' mode, installing its 
    source into $PWD/gh_clones
    """
    assert not is_installed('quail')
    dest_path = Path('gh_clones/quail').resolve()
    assert not dest_path.is_dir()
    smuggle quail    # pip: -e git+https://github.com/ContextLab/quail.git@v0.2.0#egg=quail --src gh_clones
    assert dest_path.is_dir()
    assert str(dest_path) in sys.path
    assert isinstance(quail, types.ModuleType)
    assert hasattr(quail, 'Egg')
    assert is_installed('quail==0.2.0')
    expected_cache_key = 'pip;-e;git+https://github.com/ContextLab/quail.git@v0.2.0#egg=quail;--src;gh_clones'
    assert davos.config.smuggled['quail'] == expected_cache_key, (
        f"Expected: '{expected_cache_key}'\nFound: "
        f"{davos.config.smuggled['quail']}"
    )

In [ ]:
@mark.timeout(300)
def test_smuggle_github_subdirectory():
    """
    smuggle a package tthat exists as a subdirectory of a larger GitHub 
    repository
    """
    assert not is_installed('sherlock_helpers')
    with capture_ipython_display():
        # displays an IPython object on import, which 
        # contextlib.redirect_stdout doesn't catch
        smuggle sherlock_helpers    # pip: git+https://github.com/ContextLab/sherlock-topic-model-paper.git@v1.0#subdirectory=code/sherlock_helpers
    assert isinstance(sherlock_helpers, types.ModuleType)
    assert hasattr(sherlock_helpers, 'github_url')
    assert sherlock_helpers.__version__ == '0.0.1'
    assert is_installed('sherlock_helpers==0.0.1')
    expected_cache_key = 'pip;git+https://github.com/ContextLab/sherlock-topic-model-paper.git@v1.0#subdirectory=code/sherlock_helpers'
    assert davos.config.smuggled['sherlock_helpers'] == expected_cache_key, (
        f"Expected: '{expected_cache_key}'\nFound: "
        f"{davos.config.smuggled['sherlock_helpers']}"
    )

In [ ]:
run_tests()